#### Dilated Convolutions

* Dilated convolutions introduce another parameter to convolutional layers called the **dilation rate**. 
* This defines a spacing between the values in a kernel. 
* A 3x3 kernel with a dilation rate of 2 will have the same field of view as a 5x5 kernel, while only using 9 parameters. 
* Imagine taking a 5x5 kernel and deleting every second column and row.
* This delivers a wider field of view at the same computational cost. 
* Dilated convolutions are particularly popular in the field of real-time segmentation. 
* Use them if you need a wide field of view and cannot afford multiple convolutions or larger kernels (due to computational costs).

#### Example: A dilated convolution with a dilation rate of 2 and no padding
<img src = '../pics/dilatedconv3x3rate2nopad.gif'>

### Formal Definitions
* Let $\mathit{F} : \mathbb{Z}^2 \rightarrow \mathbb{R}$ be a discrete function.
* Let $\Omega _r = [-r,r]^2 \cap\mathbb{Z}^2$ and let $\mathit{k} : \Omega_r \rightarrow \mathbb{R}$ be a discrete filter of size $(2r + 1)^2$. The discrete convolution operator $\ast$ can be defined as:
$$$$
$$(\mathit{F}\ast\mathit{k})(\mathit{p}) = \sum_{s+t=p}\mathit{F(s)k(t)}$$
$$$$
* We now generalize this operator. Let $\mathit{l}$ be a dilation factor and let $\ast_{\mathit{l}}$ be defined as:
$$(\mathit{F}\ast_{\mathit{l}}\mathit{k})(\mathit{p}) = \sum_{s+\mathit{l}t=p}\mathit{F(s)k(t)}$$
$$$$
* We will refer to $\ast_{\mathit{l}}$ as a dilated convolution or an $\mathit{l}$-dilated convolution. 
* The familiar discrete convolution $\ast$ is simply the 1-dilated convolution.

<img src='../pics/dilated_conv_124.png'>

#### What are dilation factors? Basically, the amount of space between kernel elements.
* Blue numbers in the image below represent the dilation factors applied to the Kernel
<img src='..\pics\dilation_factor.png'>

### Doing dilated convolution in TensorFlow
#### $\cdot$ Either set dilation in `tf.nn.conv2d()` or use `tf.nn.atrous_conv2d()`

In [9]:
import tensorflow as tf
import numpy as np,sys
from  scipy.signal import convolve2d

np.random.seed(678)
tf.set_random_seed(6789)
sess = tf.Session()

### Create a matrix to perform our experiments on

In [10]:
mat_size = 10
matrix = np.zeros((mat_size,mat_size)).astype(np.float32) 

for x in range(4,7):
    for y in range(3,6):
        matrix[y,x] = 1

### Create a Kernel 

In [11]:
kernel = np.array([
    [1,2,3],
    [4,5,6],
    [7,8,9]
]).astype(np.float32) 

### ====== Original Set Up ======

In [12]:
print("Matrix Shape : ",matrix.shape)
print(matrix)
print()
print("kernel Shape : ",kernel.shape)
print(kernel)

====== Original Set Up ======
Matrix Shape :  (10, 10)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

kernel Shape :  (3, 3)
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]


### ========== EXAMPLE 1 - Dilation Factor 1 ===========
* This is what we think of as a "normal" convolution

In [13]:
print("\n====== Dilated Kernel 1 ======")
print('Kernal For "Familiar" Convolution for Numpy: \n',kernel)
print()
print("========== Numpy 'familiar' Convolution Results ===============")
np_results = convolve2d(matrix,kernel,mode='valid')
print("Numpy Results Shape: ",np_results.shape)
print(np_results)

print()
print("========== Tensorfow Conv2D Results ===============")
tf_operation1_1 = tf.nn.conv2d(np.expand_dims(np.expand_dims(matrix,axis=0),axis=3),
                            np.expand_dims(np.expand_dims(np.rot90(kernel,2),axis=2),axis=3),
                            strides=[1,1,1,1],padding="VALID",
                            dilations=[1,1,1,1])

tf_result = sess.run(tf_operation1_1)

print()
print("Tensorfow Conv2D Results Shape: ",tf_result.shape)
print(np.squeeze(tf_result))


====== Dilated Kernel 1 ======
Kernal For "Familiar" Convolution for Numpy: 
 [[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]

========== Numpy 'familiar' Convolution Results ===============
Numpy Results Shape:  (8, 8)
[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  3.  6.  5.  3.  0.]
 [ 0.  0.  5. 12. 21. 16.  9.  0.]
 [ 0.  0. 12. 27. 45. 33. 18.  0.]
 [ 0.  0. 11. 24. 39. 28. 15.  0.]
 [ 0.  0.  7. 15. 24. 17.  9.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]

========== Tensorfow Conv2D Results ===============

Tensorfow Conv2D Results Shape:  (1, 8, 8, 1)
[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  3.  6.  5.  3.  0.]
 [ 0.  0.  5. 12. 21. 16.  9.  0.]
 [ 0.  0. 12. 27. 45. 33. 18.  0.]
 [ 0.  0. 11. 24. 39. 28. 15.  0.]
 [ 0.  0.  7. 15. 24. 17.  9.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]


### ========== Tensorfow Atrous Conv2D Results ===============
* "Atrous convolution" = a convolution with holes, or a dilated convolution

In [14]:
tf_operation1_2 = tf.nn.atrous_conv2d(np.expand_dims(np.expand_dims(matrix,axis=0),axis=3),
                                    np.expand_dims(np.expand_dims(np.rot90(kernel,2),axis=2),axis=3),
                                    rate=1,padding="VALID")
tf_result = sess.run(tf_operation1_2)
print("Tensorfow Atrous Results Shape: ",tf_result.shape)
print(np.squeeze(tf_result))

Tensorfow Atrous Results Shape:  (1, 8, 8, 1)
[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  3.  6.  5.  3.  0.]
 [ 0.  0.  5. 12. 21. 16.  9.  0.]
 [ 0.  0. 12. 27. 45. 33. 18.  0.]
 [ 0.  0. 11. 24. 39. 28. 15.  0.]
 [ 0.  0.  7. 15. 24. 17.  9.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]


### ========== EXAMPLE 2 - Dilation Factor 2 ===========
* Notice that now, while the TensorFlow operations continue to use the original 3x3 kernel as we change the dilation factor
* However, in Numpy we are manually adjusting the kernel to reflect the dilation, but using the same `convolve2d()` numpy function.

In [21]:
print("\n====== Dilated Kernel For Dilation Factor 2 ======")
kernel_dilation2 = np.array([
    [1,0,2,0,3],
    [0,0,0,0,0],
    [4,0,5,0,6],
    [0,0,0,0,0],
    [7,0,8,0,9]
])
print(kernel_dilation2)

print()
print("========== Numpy 'familiar' Convolution Results ===============")
np_results = convolve2d(matrix,kernel_dilation2,mode='valid')
print("Numpy Results Shape: ",np_results.shape)
print(np_results)

print()
print("========== Tensorfow Conv2D Results ===============")
tf_operation2_1 = tf.nn.conv2d(np.expand_dims(np.expand_dims(matrix,axis=0),axis=3),
                            np.expand_dims(np.expand_dims(np.rot90(kernel,2),axis=2),axis=3),
                            strides=[1,1,1,1],padding="VALID",
                            dilations=[1,2,2,1])
tf_result = sess.run(tf_operation2_1)
print("Tensorfow Conv2D Results Shape: ",tf_result.shape)
print(np.squeeze(tf_result))

print()
print("========== Tensorfow Atrous Conv2D Results ===============")
tf_operation2_2 = tf.nn.atrous_conv2d(np.expand_dims(np.expand_dims(matrix,axis=0),axis=3),
                                    np.expand_dims(np.expand_dims(np.rot90(kernel,2),axis=2),axis=3),
                                    rate=2,padding="VALID")
tf_result = sess.run(tf_operation2_2)
print("Tensorfow Atrous Results Shape: ",tf_result.shape)
print(np.squeeze(tf_result))


====== Dilated Kernel For Dilation Factor 2 ======
[[1 0 2 0 3]
 [0 0 0 0 0]
 [4 0 5 0 6]
 [0 0 0 0 0]
 [7 0 8 0 9]]

========== Numpy 'familiar' Convolution Results ===============
Numpy Results Shape:  (6, 6)
[[ 1.  1.  3.  2.  5.  3.]
 [ 5.  5. 12.  7. 16.  9.]
 [ 4.  4.  9.  5. 11.  6.]
 [11. 11. 24. 13. 28. 15.]
 [ 7.  7. 15.  8. 17.  9.]
 [ 7.  7. 15.  8. 17.  9.]]

========== Tensorfow Conv2D Results ===============
Tensorfow Conv2D Results Shape:  (1, 6, 6, 1)
[[ 1.  1.  3.  2.  5.  3.]
 [ 5.  5. 12.  7. 16.  9.]
 [ 4.  4.  9.  5. 11.  6.]
 [11. 11. 24. 13. 28. 15.]
 [ 7.  7. 15.  8. 17.  9.]
 [ 7.  7. 15.  8. 17.  9.]]

========== Tensorfow Atrous Conv2D Results ===============
Tensorfow Atrous Results Shape:  (1, 6, 6, 1)
[[ 1.  1.  3.  2.  5.  3.]
 [ 5.  5. 12.  7. 16.  9.]
 [ 4.  4.  9.  5. 11.  6.]
 [11. 11. 24. 13. 28. 15.]
 [ 7.  7. 15.  8. 17.  9.]
 [ 7.  7. 15.  8. 17.  9.]]


### ========== EXAMPLE 3 - Dilation Factor 3 ===========

In [22]:
print("\n====== Dilated Kernel for Dilation Factor 3 ======")
kernel_dilation3 = np.array([
    [1,0,0,2,0,0,3],
    [0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0],
    [4,0,0,5,0,0,6],
    [0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0],
    [7,0,0,8,0,0,9]
])
print(kernel_dilation3)

print()
print("========== Numpy 'familiar' Convolution Results ===============")
np_results = convolve2d(matrix,kernel_dilation3,mode='valid')
print("Numpy Results Shape: ",np_results.shape)
print(np_results)

print()
print("========== Tensorfow Conv2D Results ===============")
tf_operation4_1 = tf.nn.conv2d(np.expand_dims(np.expand_dims(matrix,axis=0),axis=3),
                            np.expand_dims(np.expand_dims(np.rot90(kernel,2),axis=2),axis=3),
                            strides=[1,1,1,1],padding="VALID",
                            dilations=[1,3,3,1])

tf_result = sess.run(tf_operation4_1)

print("Tensorfow Conv2D Results Shape: ",tf_result.shape)
print(np.squeeze(tf_result))

print()
print("========== Tensorfow Atrous Conv2D Results ===============")
tf_operation4_2 = tf.nn.atrous_conv2d(np.expand_dims(np.expand_dims(matrix,axis=0),axis=3),
                                    np.expand_dims(np.expand_dims(np.rot90(kernel,2),axis=2),axis=3),
                                    rate=3,padding="VALID")
tf_result = sess.run(tf_operation4_2)
print("Tensorfow Atrous Results Shape: ",tf_result.shape)
print(np.squeeze(tf_result))


====== Dilated Kernel for Dilation Factor 3 ======
[[1 0 0 2 0 0 3]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [4 0 0 5 0 0 6]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [7 0 0 8 0 0 9]]

========== Numpy 'familiar' Convolution Results ===============
Numpy Results Shape:  (4, 4)
[[4. 5. 5. 5.]
 [4. 5. 5. 5.]
 [4. 5. 5. 5.]
 [7. 8. 8. 8.]]

========== Tensorfow Conv2D Results ===============
Tensorfow Conv2D Results Shape:  (1, 4, 4, 1)
[[4. 5. 5. 5.]
 [4. 5. 5. 5.]
 [4. 5. 5. 5.]
 [7. 8. 8. 8.]]

========== Tensorfow Atrous Conv2D Results ===============
Tensorfow Atrous Results Shape:  (1, 4, 4, 1)
[[4. 5. 5. 5.]
 [4. 5. 5. 5.]
 [4. 5. 5. 5.]
 [7. 8. 8. 8.]]


### ========== EXAMPLE 4 - Dilation Factor 4 ===========

In [23]:
print("\n====== Dilated Kernel for Dilation Factor 4 ======")
kernel_dilation4 = np.array([
    [1,0,0,0,2,0,0,0,3],
    [0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0],
    [4,0,0,0,5,0,0,0,6],
    [0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0],
    [7,0,0,0,8,0,0,0,9]
])
print(kernel_dilation4)

print()
print("========== Numpy 'familiar' Convolution Results ===============")
np_results = convolve2d(matrix,kernel_dilation4,mode='valid')
print("Numpy Results Shape: ",np_results.shape)
print(np_results)

print()
print("========== Tensorfow Conv2D Results ===============")
tf_operation4_1 = tf.nn.conv2d(np.expand_dims(np.expand_dims(matrix,axis=0),axis=3),
                            np.expand_dims(np.expand_dims(np.rot90(kernel,2),axis=2),axis=3),
                            strides=[1,1,1,1],padding="VALID",
                            dilations=[1,4,4,1])
tf_result = sess.run(tf_operation4_1)
print("Tensorfow Conv2D Results Shape: ",tf_result.shape)
print(np.squeeze(tf_result))

print()
print("========== Tensorfow Atrous Conv2D Results ===============")
tf_operation4_2 = tf.nn.atrous_conv2d(np.expand_dims(np.expand_dims(matrix,axis=0),axis=3),
                                    np.expand_dims(np.expand_dims(np.rot90(kernel,2),axis=2),axis=3),
                                    rate=4,padding="VALID")
tf_result = sess.run(tf_operation4_2)
print("Tensorfow Atrous Results Shape: ",tf_result.shape)
print(np.squeeze(tf_result))


====== Dilated Kernel for Dilation Factor 4 ======
[[1 0 0 0 2 0 0 0 3]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [4 0 0 0 5 0 0 0 6]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [7 0 0 0 8 0 0 0 9]]

========== Numpy 'familiar' Convolution Results ===============
Numpy Results Shape:  (2, 2)
[[5. 5.]
 [5. 5.]]

========== Tensorfow Conv2D Results ===============
Tensorfow Conv2D Results Shape:  (1, 2, 2, 1)
[[5. 5.]
 [5. 5.]]

========== Tensorfow Atrous Conv2D Results ===============
Tensorfow Atrous Results Shape:  (1, 2, 2, 1)
[[5. 5.]
 [5. 5.]]


### ====== Set Up With Dilation Factor of 4 ======

In [27]:
print("Matrix Shape : ",matrix.shape)
print(matrix)
print()
print("kernel Shape : ",kernel_dilation4.shape)
print(kernel_dilation4)

Matrix Shape :  (10, 10)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

kernel Shape :  (9, 9)
[[1 0 0 0 2 0 0 0 3]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [4 0 0 0 5 0 0 0 6]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [7 0 0 0 8 0 0 0 9]]
